# Importing the libraries

In [1]:
import numpy as np
from collections import OrderedDict
import os
import torch
from torch.utils.data import random_split
from torch import nn
from torchvision import transforms, models, datasets

# Importing the dataset

In [2]:
DATASET_DIRECTORY = '../Datasets/Intel/'
TRAINING_PATH = DATASET_DIRECTORY + 'seg_train/seg_train/'
TEST_PATH = DATASET_DIRECTORY + 'seg_test/seg_test/'

train_cat_counts = {}
for cat in os.listdir(TRAINING_PATH):
    counts = len(os.listdir(os.path.join(TRAINING_PATH, cat)))
    train_cat_counts[cat] = counts

test_cat_counts = {}
for cat in os.listdir(TEST_PATH):
    counts = len(os.listdir(os.path.join(TEST_PATH, cat)))    
    test_cat_counts[cat] = counts

print("Size of the training set:", sum(train_cat_counts.values()))    
print("Label frequencies of the training set:", train_cat_counts)
print(10*'-')
print("Size of the test set:", sum(test_cat_counts.values()))
print("Label frequencies of the test set:", test_cat_counts)

Size of the training set: 14034
Label frequencies of the training set: {'buildings': 2191, 'forest': 2271, 'glacier': 2404, 'mountain': 2512, 'sea': 2274, 'street': 2382}
----------
Size of the test set: 3000
Label frequencies of the test set: {'buildings': 437, 'forest': 474, 'glacier': 553, 'mountain': 525, 'sea': 510, 'street': 501}


# Splitting & Preprocessing & Loading

In [3]:
random_seed = 42
batch_size = 64
# mean and std which for ResNet50
mean = [0.485, 0.456, 0.406] 
std = [0.229, 0.224, 0.225]

train_transforms = transforms.Compose([transforms.Resize((150, 150)), # Resize all images 
                                       transforms.RandomResizedCrop(150),# Crop
                                       transforms.RandomRotation(30), # Rotate 
                                       transforms.RandomHorizontalFlip(), # Flip
                                       transforms.ToTensor(), # Convert
                                       transforms.Normalize(torch.Tensor(mean), torch.Tensor(std)) # Normalize
                                       ])



test_transforms = transforms.Compose([transforms.Resize((150, 150)),
                                     transforms.CenterCrop(150),
                                     transforms.ToTensor(),
                                     transforms.Normalize(torch.Tensor(mean),torch.Tensor(std))
                                     ])

# Tmp torchvision datasets.Image folder to split into train and validation sets
tmp_data = datasets.ImageFolder(TRAINING_PATH, transform=train_transforms)
# len(tmp_data): 14034

# Randomsplit tmp data based on length of dataset and set seed for reproducable split
train_data, val_data = random_split(tmp_data, [10000, 4034], generator=torch.Generator().manual_seed(random_seed))
# Test set with with test transforms 
test_data = datasets.ImageFolder(TEST_PATH, transform=test_transforms)


# Set Pytorch dataloaders, batch_size, training set shuffle
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=batch_size)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

# VGG16 pretrained on ImageNet

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
def init(random_seed): 
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)

def create_model():
    random_seed = 42
    init(random_seed)
    vgg = models.vgg16(pretrained=True)
    # Freeze model params 
    for param in vgg.parameters():
        param.requires_grad = False
    vgg.classifier[6] = nn.Linear(in_features=4096, out_features=6)    
    vgg.to(device)
    return vgg    

## The train() & evalaute() functions: 
The evalute() function returns the avg model loss on the validation set.   

In [5]:
import gc
from torch.nn import Softmax
from torch.optim import Adam, AdamW

def train(model, loss_fn, optimizer):
    model.train()
    total_loss = 0
    total_len = len(train_dataloader)
    softmax_func = Softmax(dim=1)

    for i, batch in enumerate(train_dataloader):
        step = i+1
        percent = "{0:.5f}".format(100 * (step / float(total_len)))
        lossp = "{0:.5f}".format(total_loss/step)
        filledLength = int(100 * step // total_len)
        bar = '█' * filledLength + '>'  *(filledLength < 100) + '.' * (99 - filledLength)
        print(f'\rBatch {step}/{total_len} |{bar}| {percent}% complete, loss={lossp}', end='')
        imgs, labels = batch[0].to(device), batch[1].to(device)
        del batch
        gc.collect()
        torch.cuda.empty_cache()
        model.zero_grad()
        preds = model(imgs)
        preds = softmax_func(preds) 
        loss = loss_fn(preds.double(), labels.double())
        total_loss += float(loss.item())
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

    gc.collect()
    torch.cuda.empty_cache()

    avg_loss = total_loss / total_len
    
    return avg_loss

def evaluate(model, loss_fn):
    val_loss = 0
    softmax_func = Softmax(dim=1)
    model.eval()
    with torch.no_grad():
        for batch in val_dataloader:
            imgs, labels = batch[0].to(device), batch[1].to(device)
            gc.collect()
            torch.cuda.empty_cache()
            preds = model(imgs)
            preds = softmax_func(preds)
            loss = loss_fn(preds.double(), labels.double())
            val_loss += loss.item()
        return  val_loss / len(val_dataloader)

# Loss functions

$$ \ell_{CE}(\mathrm{\hat{y}}, \mathrm{y}) = -\sum_{c=1}^{C} y_c\log(\hat{y}_c),\,\, \ell_{RJM}(\mathrm{\hat{y}}, \mathrm{y}) = \sum_{c=1}^{C} y_c(1-\sqrt{\hat{y}_c}). $$

In [6]:
def CE(y_hat, y):
    return torch.sum(-1*torch.log(y_hat[range(y.size()[0]), y.long()]) / y.size()[0])

def RJM(y_hat, y):
    return torch.sum(1 - torch.sqrt(y_hat[range(y.size()[0]), y.long()])) / y.size()[0]

# Optimizer: SGD

## CE loss

In [8]:
from torch.optim import SGD
vgg_sgd_ce = create_model()
history_vgg_sgd_ce = []
optimizer = SGD(vgg_sgd_ce.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10,15], gamma=0.2)
loss_fn = CE
epochs = 20
current = 1
# for each epoch
while current <= epochs:

    print(f'\nEpoch {current} / {epochs}:')

    # train model
    train_loss = train(model=vgg_sgd_ce, loss_fn = loss_fn, optimizer=optimizer)

    # evaluate model
    val_loss = evaluate(model=vgg_sgd_ce, loss_fn=loss_fn)
    
    scheduler.step()
    
    print(f'\n\nTraining Loss: {train_loss:.5f}')
    print(f'Val Loss: {val_loss:.5f}')

    history_vgg_sgd_ce.append((train_loss, val_loss))

    current = current + 1


Epoch 1 / 20:
Batch 157/157 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% complete, loss=1.73250

Training Loss: 1.74232
Val Loss: 1.55337

Epoch 2 / 20:
Batch 157/157 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% complete, loss=1.46992

Training Loss: 1.47810
Val Loss: 1.29958

Epoch 3 / 20:
Batch 157/157 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% complete, loss=1.21464

Training Loss: 1.22149
Val Loss: 1.05906

Epoch 4 / 20:
Batch 157/157 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% complete, loss=1.01528

Training Loss: 1.02206
Val Loss: 0.88318

Epoch 5 / 20:
Batch 157/157 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% complete, loss=0.87752

T

### Save the model

In [17]:
torch.save(vgg_sgd_ce.state_dict(), './checkpoints/vgg_sgd_ce.pth')

## RJM loss

In [10]:
vgg_sgd_rjm = create_model()
history_vgg_sgd_rjm = []
optimizer = SGD(vgg_sgd_rjm.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10,15], gamma=0.2)
loss_fn = RJM
epochs = 20
current = 1
# for each epoch
while current <= epochs:

    print(f'\nEpoch {current} / {epochs}:')

    # train model
    train_loss = train(model=vgg_sgd_rjm, loss_fn = loss_fn, optimizer=optimizer)

    # evaluate model
    val_loss = evaluate(model=vgg_sgd_rjm, loss_fn=loss_fn)
    
    scheduler.step()
    
    print(f'\n\nTraining Loss: {train_loss:.5f}')
    print(f'Val Loss: {val_loss:.5f}')

    history_vgg_sgd_rjm.append((train_loss, val_loss))

    current = current + 1


Epoch 1 / 20:
Batch 157/157 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% complete, loss=0.56717

Training Loss: 0.57048
Val Loss: 0.53163

Epoch 2 / 20:
Batch 157/157 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% complete, loss=0.49889

Training Loss: 0.50178
Val Loss: 0.45738

Epoch 3 / 20:
Batch 157/157 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% complete, loss=0.42040

Training Loss: 0.42273
Val Loss: 0.37576

Epoch 4 / 20:
Batch 157/157 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% complete, loss=0.35078

Training Loss: 0.35304
Val Loss: 0.30936

Epoch 5 / 20:
Batch 157/157 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% complete, loss=0.29680

T

### Save the model

In [18]:
torch.save(vgg_sgd_rjm.state_dict(), './checkpoints/vgg_sgd_rjm.pth')

## Evaluation

In [15]:
from sklearn.metrics import accuracy_score, f1_score
y_true_sgd_ce = np.array([])
y_pred_sgd_ce = np.array([])
vgg_sgd_ce.eval()
with torch.no_grad():
    for batch in val_dataloader:
        imgs, labels = batch[0].to(device), batch[1].to(device)
        preds = vgg_sgd_ce(imgs).detach().cpu().numpy()
        labels = labels.detach().cpu().numpy()
        preds_indices = preds.argmax(axis=1)
        y_true_sgd_ce = np.concatenate((y_true_sgd_ce, labels))
        y_pred_sgd_ce = np.concatenate((y_pred_sgd_ce, preds_indices))
    
print('VGG16: Optimzier = SGD, Loss = CE')
print(10*'-')    
print('Accuracy: {0:0.4f}'.format(accuracy_score(y_true_sgd_ce, y_pred_sgd_ce)))
print('Macro F1: {0:0.4f}'.format(f1_score(y_true_sgd_ce, y_pred_sgd_ce, average='macro')))

VGG16: Optimzier = SGD, Loss = CE
----------
Accuracy: 0.7955
Macro F1: 0.7959


In [14]:
y_true_sgd_rjm = np.array([])
y_pred_sgd_rjm = np.array([])
vgg_sgd_rjm.eval()
with torch.no_grad():
    for batch in val_dataloader:
        imgs, labels = batch[0].to(device), batch[1].to(device)
        preds = vgg_sgd_rjm(imgs).detach().cpu().numpy()
        labels = labels.detach().cpu().numpy()
        preds_indices = preds.argmax(axis=1)
        y_true_sgd_rjm = np.concatenate((y_true_sgd_rjm, labels))
        y_pred_sgd_rjm = np.concatenate((y_pred_sgd_rjm, preds_indices))
    
print('VGG16: Optimzier = SGD, Loss = RJM')
print(10*'-')    
print('Accuracy: {0:0.4f}'.format(accuracy_score(y_true_sgd_rjm, y_pred_sgd_rjm)))
print('Macro F1: {0:0.4f}'.format(f1_score(y_true_sgd_rjm, y_pred_sgd_rjm, average='macro')))

VGG16: Optimzier = SGD, Loss = RJM
----------
Accuracy: 0.7965
Macro F1: 0.7979


In [12]:
history_vgg_sgd_ce

[(1.7423240135226725, 1.5533659691856252),
 (1.4780956886157184, 1.2995807431223263),
 (1.2214885981900583, 1.0590605267544841),
 (1.0220575068178843, 0.8831785307282952),
 (0.884064616657325, 0.7559370820042073),
 (0.7811117078371027, 0.6946927015386313),
 (0.7302156766935419, 0.6440735018382869),
 (0.6846914576546775, 0.6108974151802413),
 (0.6695676376204002, 0.586160217179046),
 (0.643740174364163, 0.578380042515941),
 (0.646376258585805, 0.5726090452268723),
 (0.600983994311819, 0.5728806836768332),
 (0.6275388569137805, 0.557446680204475),
 (0.6195425294881943, 0.5708087187024208),
 (0.6128390177469346, 0.5622517098801383),
 (0.6125574280773124, 0.5524263972208142),
 (0.6112178223597535, 0.5717911694124375),
 (0.6198585868733744, 0.5755721611791537),
 (0.6092478541808485, 0.5369680360973882),
 (0.6137096162312751, 0.5677567470918513)]

In [13]:
history_vgg_sgd_rjm

[(0.5704775954566864, 0.5316340580370936),
 (0.5017810324127187, 0.4573766043098394),
 (0.4227251482536649, 0.37575730192327406),
 (0.35303814075914297, 0.30936492929492004),
 (0.2990021932489004, 0.25858460376353537),
 (0.2594500796335141, 0.2312666115211531),
 (0.23688732374531754, 0.20983554466250665),
 (0.21971649039243407, 0.19940541755147845),
 (0.21052069261041553, 0.18610575934721577),
 (0.20129233437020352, 0.1851157993960144),
 (0.2001821538844043, 0.1803834613812447),
 (0.187417129825905, 0.1813401213039784),
 (0.19451413933755618, 0.17786807610848623),
 (0.1925444589187849, 0.1803185357622928),
 (0.19126707473123966, 0.17971547030658158),
 (0.19017836600713112, 0.1748474558278659),
 (0.19163240996089106, 0.18322362421806826),
 (0.19035898576771595, 0.18261689694455674),
 (0.18828590544896498, 0.16885118525259896),
 (0.18826672383058943, 0.17995293623837771)]